# word2vec
word2vecにはCBOWとskip-gramが存在。  
CBOWはコンテキストから単語を推測、skip-gramは単語から周囲のコンテキストを推測。


### CBOW
CBOWモデルはコンテキストからターゲットを推測するモデル。  
（前後1単語から推測する場合は）入力層が二つ、中間層が一つ、出力層で出来ている。  
出力層は各単語のスコアを計算し、最終的にsoftmax関数を用いてターゲットとなる単語を選ぶ。  
損失関数は
$$ L = -\frac{1}{T}\sum^T_{t=1}\log{P(w_t | w_{t-1}, w_{t+1})}$$


### skip-gram
CBOWモデルを逆転させたような構造。  
前後1単語を予測する場合は次のような損失関数
$$ L= -\frac{1}{T}\sum^T_{t=1}(\log{P(w_{t-1}|w_t)+log{P(w_{t+1}|w_t)}}) $$


参考url
- https://qiita.com/g-k/items/69afa87c73654af49d36


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random


In [ ]:
corpus = "You say goodbye and I say hello"
# print(corpus.lower())
corpus = corpus.lower().split()
# print(corpus)
# print(set(corpus))
# print(enumerate(set(corpus)))
# for i, item in enumerate(set(corpus)):
#     print(i, item)
# 単語をIDに変換
word2idx = {word: idx for idx, word in enumerate(set(corpus))} # wordからindexを取得するリスト
print(word2idx)
idx2word = {idx: word for word, idx in word2idx.items()} # indexからwordを取得するリスト
print(word2idx.items())
print(idx2word)
vocab_size = len(word2idx)

# 単語ID列
corpus_ids = [word2idx[w] for w in corpus]
print("\"You say goodbye and I say hello\" is transformed as", corpus_ids)


0 say
1 i
2 hello
3 you
4 goodbye
5 and
{'say': 0, 'i': 1, 'hello': 2, 'you': 3, 'goodbye': 4, 'and': 5}
dict_items([('say', 0), ('i', 1), ('hello', 2), ('you', 3), ('goodbye', 4), ('and', 5)])
{0: 'say', 1: 'i', 2: 'hello', 3: 'you', 4: 'goodbye', 5: 'and'}
"You say goodbye and I say hello" is transformed as [3, 0, 4, 5, 1, 0, 2]


In [4]:
def generate_training_data(corpus_ids, window_size=2):
    pairs = []
    for i, target in enumerate(corpus_ids):
        for j in range(-window_size, window_size + 1):
            if j == 0 or i + j < 0 or i + j >= len(corpus_ids):
                continue
            context = corpus_ids[i + j]
            pairs.append((target, context))
    return pairs

pairs = generate_training_data(corpus_ids)


In [30]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.input_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.output_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center_words, context_words, negative_words):
        center_embeds = self.input_embeddings(center_words)
        pos_embeds = self.output_embeddings(context_words)
        neg_embeds = self.output_embeddings(negative_words)

        pos_score = torch.sum(center_embeds * pos_embeds, dim=1)
        pos_loss = torch.log(torch.sigmoid(pos_score))

        neg_score = torch.bmm(neg_embeds, center_embeds.unsqueeze(2)).squeeze(2)
        neg_loss = torch.log(torch.sigmoid(-neg_score), dim=1)

        return -(pos_loss + neg_loss).mean()


In [31]:
embedding_dim = 10
model = Word2Vec(vocab_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ネガティブサンプリング関数
def get_negative_samples(pos_word, vocab_size, num_neg=5):
    neg_samples = []
    while len(neg_samples) < num_neg:
        w = random.randint(0, vocab_size - 1)
        if w != pos_word:
            neg_samples.append(w)
    return neg_samples

# 学習
for epoch in range(100):
    total_loss = 0
    for center, context in pairs:
        center_tensor = torch.tensor([center])
        context_tensor = torch.tensor([context])
        negative_tensor = torch.tensor([get_negative_samples(context, vocab_size)]).unsqueeze(0)

        loss = model(center_tensor, context_tensor, negative_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")


RuntimeError: batch1 must be a 3D tensor

In [8]:
embeddings = model.input_embeddings.weight.data
for word, idx in word2idx.items():
    print(f"{word}: {embeddings[idx]}")


say: tensor([-0.2802,  0.8585,  0.8102,  0.5385, -0.0954,  1.1551,  0.2458, -1.8385,
        -0.3713, -1.2752])
i: tensor([ 0.2179,  0.2700,  0.7094,  1.1787, -0.3879,  0.6546, -0.9337,  0.7479,
        -0.1273, -0.7252])
hello: tensor([-1.4959, -0.6925, -0.4337, -1.3229, -0.2961, -0.3127,  1.0085,  0.4916,
         0.7628, -1.1374])
you: tensor([-0.1185, -0.3650, -0.9618, -1.0986,  0.5777,  0.4606,  0.1865,  0.1715,
        -0.1041,  1.1317])
goodbye: tensor([ 0.0953, -1.0774, -0.5591,  0.3436, -0.1011,  0.8020, -0.0540, -0.0117,
         0.1995,  0.7604])
and: tensor([ 0.2971, -0.3863, -0.0771, -1.1469,  0.8770, -1.3631, -2.7957,  0.3792,
        -0.2224,  0.0461])
